In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.linalg as la
import matplotlib.pyplot as plt

from tqdm import tqdm
from IPython.display import clear_output
from scipy.stats import norm,uniform,multivariate_normal
np.random.seed(10)

### Markov Chain Monte-Carlo using Hamiltonian Dynamics

###### Metripolis Hasting algorithm can stuck into one of the modals of the distribution and not expling the rest of the modes.
###### One way to circumvent this my heuristically adjusting the step-size using the spread of the proptoosal distribution. However too large step size leads to a large number of rejections, while a too small step-size makes makes the exploration too slow.
###### In high-dimensional space the exploration is nenarly a random-walk behavior thus the exploration is sup-optimal.

To mitigate these drawbacks Hamitonian Monte-Carlo (HMC) utilizes the target distribution and the laws of dynamics in mechanical physics to design adaptive step-size for the proptoosed samples.

The target distribution p(z) is then a modeled using the Gibbs canonical distribution from statistical mechanics as
$$p(z)\propto e^{\frac{-U(z)}{T}} $$ where T is the temperature and U(z) is the energy of the state for the particle at state z.

Apart from the potential energy U(z) this method introduces an additional auxilliary  component kinetic energy K(v) that is dependent on the speed (v) as auxilliary variable.

Eventually the total mechanical energy is:
$$E(z,v)=U(z)+K(v),s.t: K(v)=\sum_{i}\frac{v_{i}^2}{2}$$

The state distribution of the particles is then dependent on the total energy as:

$$p(z,v)\propto e^{\frac{-E(z,v)}{T}}=e^{\frac{-U(z)}{T}}e^{\frac{-K(v)}{T}}\propto p(z)p(v)$$

#### The physicial dynamics of the target distribution through Hamiltonian

In order to sample multiple different positions of the samples inside the energy well defined from E(z,v) we utilize these two physics equations:
$$\frac{\partial z_{i}(t)}{\partial t}=\frac{\partial E(z,v)}{\partial v_{i}}=\frac{\partial K(z)}{\partial v_{i}}$$
$$m\frac{\partial v_{i}(t)}{\partial t}=-\frac{\partial E(z,v)}{\partial z_{i}}=-\frac{\partial U(z)}{\partial z_{i}} $$

Since the energy of the closed system is preserved $E(z,v)=E_{0}$ it is possible to get different samples inside this target distribution while simulating particles whose statistical trajectory is guided by the two equations above.

Sampling the speed (v) is quite simple as it follows a (multivariate) normal distribution:

$$p(v)=e^{\frac{-K(v)}{T}}=e^{\frac{-\sum_{i}mv_{i}}{2T}}=e^{\frac{-mV^{T}V}{2T}}$$

###### In a nutshell

Start the sample moving with a random speed drawn from the normal distribution and stop it.
Continue this proceedure until the sufficient number of samples have been accummulated.

However, numerical solutions to the partial derivative equations (PDE) cannot be solved analytically and their numerical solution does not ensure the preservation of the energy $E(z,v)$.

To mitigate this problem Metropolis Hastings rejections are employed to compensate difference in energy between energy between the start the and the stop of the particle position.

Leapfrog numerical integration offers an numerical integration that is reversible in time.
This reversability ensures the detailed balance.

###### Physical analogy

The trajectory of the particle that roams inside the energy well defined by the target distribution is equivalent to a classical harmonic oscilator without any dampling (conservation of energy).
This is governt by a second had ordinary differential equation (ODE) $z^{''}+x=0$.
To simplify the solution this is converted into two ODEs where $z^{'}=v$.

$$z^{'}=v $$
$$v^{'}=-z $$

These two equations equivalues to a system with kinetic energy $K(v)=\frac{1}{2}v^{2}$ and potential energy $U(z)=\frac{1}{2}z^{2}$.
In the case of multidimensial distribution:
$$K(v)=\frac{v^{T}v}{2}$$
$$U(z)=\frac{z^{T}z}{2}$$

Where the acceptance rate is:

$$A(z^*,z^{(\tau)})=\frac{e^{-U(z^*)-K(z^*)}}{e^{-U(z^{(\tau)})-K(z^{(\tau)})}}=e^{U(z^{(\tau)})-U(z^*)+K(z^{(\tau)})-K(z^*)}$$

where the ODEs $z^{'}=v $, $v^{'}=-x$ can be organised into a matrix formation as $z^{'}=Ax$ where:

$A=\begin{bmatrix}
0 & 1\\
-1 & 0
\end{bmatrix} $

Since the eigenvalues of the matrix $A$ are $i$ and $-i$ the solution of position $x(t)=e^{it}$.
Equation for $x(t)$ is just a circle that does not changes its shape.

###### Euler solution to ODE

Numerical solution to the ODE $z^{'}=Ax$ is from Euler where $\frac{z_{n+1}-z_{n}}{\Delta t}=Az_{n}\to z_{n+1}=z_{n}+\Delta tAz_{n}=(I+\Delta tA)z_{n}=Bz_{n};s.t:B=I+\Delta tA$

###### Leapfrog solution to ODE
Instead of performing the updates simultaneosly, leapfrog method splits this across variables.
It makes one half-step towards the first variable.
Makes a full step towards the second variable using the updated first variable.
Takes one final half step for the first variable using the updated second variable.


###### Concrete example for HMC execution

Consider the target distribution is a multivariate Gaussian with zero mean and covariance $\Sigma$ such as $z\sim N(o,\Sigma)$ and:

$$U(z)=\frac{z^{T}\Sigma^{-1}z}{2}$$

The matrix A in this case is:


$A=\begin{bmatrix}
0 & 1\\
-\Sigma^{-1} & 0
\end{bmatrix} $


In [12]:
data=pd.read_csv('Seatbelts.csv')
print(data.to_numpy())
print(data)


[[1.00000000e+00 1.07000000e+02 1.68700000e+03 ... 1.02971812e-01
  1.20000000e+01 0.00000000e+00]
 [2.00000000e+00 9.70000000e+01 1.50800000e+03 ... 1.02362996e-01
  6.00000000e+00 0.00000000e+00]
 [3.00000000e+00 1.02000000e+02 1.50700000e+03 ... 1.02062491e-01
  1.20000000e+01 0.00000000e+00]
 ...
 [1.90000000e+02 1.20000000e+02 1.57500000e+03 ... 1.16465522e-01
  7.00000000e+00 1.00000000e+00]
 [1.91000000e+02 1.37000000e+02 1.73700000e+03 ... 1.16026113e-01
  4.00000000e+00 1.00000000e+00]
 [1.92000000e+02 1.54000000e+02 1.76300000e+03 ... 1.16066729e-01
  7.00000000e+00 1.00000000e+00]]
     Unnamed: 0  DriversKilled  drivers  front  rear    kms  PetrolPrice  \
0             1            107     1687    867   269   9059     0.102972   
1             2             97     1508    825   265   7685     0.102363   
2             3            102     1507    806   319   9963     0.102062   
3             4             87     1385    814   407  10955     0.100873   
4             5     